In [ ]:
!pip install tensorflow keras pillow scikit-learn matplotlib

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU tersedia:", gpus)
else:
    print("GPU tidak tersedia, menggunakan CPU.")

GPU tersedia: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import kagglehub

path = kagglehub.dataset_download("moltean/fruits")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fruits


In [ ]:
import os
print(os.listdir('/kaggle/input/fruits/fruits-360_original-size'))
main_data_dir = '/kaggle/input/fruits/fruits-360_original-size'
data_dir = os.path.join(main_data_dir, 'fruits-360-original-size')
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Test')

['fruits-360-original-size']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation dan normalisasi
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(500, 500),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(500, 500),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Testing generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(500, 500),
    batch_size=32,
    class_mode='categorical'
)

Found 26335 images belonging to 90 classes.
Found 2887 images belonging to 90 classes.
Found 14527 images belonging to 90 classes.


In [ ]:
import json

with open('classes.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(500, 500, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 498, 498, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 249, 249, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 247, 247, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 123, 123, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 968256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │   123,936,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 90)             │        11,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,967,898 (472.90 MB)

 Trainable params: 123,967,898 (472.90 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
with tf.device('/GPU:0'):
  history = model.fit(
      train_generator,
      validation_data=validation_generator,
      epochs=10
  )

In [ ]:
with tf.device('/GPU:0'):
  history2 = model.fit(
      train_generator,
      validation_data=validation_generator,
      epochs=30,
      initial_epoch=10
  )

Epoch 11/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 90ms/step - accuracy: 0.9743 - loss: 0.0761 - val_accuracy: 0.9744 - val_loss: 0.1065
Epoch 12/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 71s 88ms/step - accuracy: 0.9762 - loss: 0.0716 - val_accuracy: 0.9662 - val_loss: 0.1735
Epoch 13/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 90ms/step - accuracy: 0.9778 - loss: 0.0672 - val_accuracy: 0.9783 - val_loss: 0.0743
Epoch 14/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 90ms/step - accuracy: 0.9813 - loss: 0.0519 - val_accuracy: 0.9797 - val_loss: 0.0641
Epoch 15/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 74s 92ms/step - accuracy: 0.9784 - loss: 0.0660 - val_accuracy: 0.9712 - val_loss: 0.1212
Epoch 16/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 90ms/step - accuracy: 0.9803 - loss: 0.0589 - val_accuracy: 0.9783 - val_loss: 0.1188
Epoch 17/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 89ms/step - accuracy: 0.9818 - loss: 0.0582 - val_accuracy: 0.9786 - val_loss: 0.1804
Epoch 18/30
801/801 ━━━━━━━━━━━━━━━━━━━━ 72s 90ms/step - accuracy: 0.9843 - loss: 0

In [ ]:
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc*100:.2f}%")

408/408 ━━━━━━━━━━━━━━━━━━━━ 120s 294ms/step - accuracy: 0.9967 - loss: 0.0108
Test Accuracy: 99.62%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path = '/content/drive/MyDrive/uas_cv'
os.makedirs(save_path, exist_ok=True)

# Simpan model ke folder uas_cv
model_save_file = os.path.join(save_path, 'model_klasifikasi_buah.h5')
model.save(model_save_file)

print(f"Model berhasil disimpan di: {model_save_file}")

Model berhasil disimpan di: /content/drive/MyDrive/uas_cv/model_klasifikasi_buah.h5
